In [1]:
from utils import MongoConnectionManager
from pprint import pprint
import json
import os

In [16]:
def process_output(data):
    processed_data = {}
    for key, value in data.items():
        if type(value) == list:
            component_data = []
            for item in value:
                component_data.append({'name': item['name'].title(), 'value': item['value']})
            processed_data[key] = component_data
        else:
            processed_data[key] = value
    return processed_data

In [20]:
def prepare_chart_data(properties):
    database = os.environ.get('DATABASE')
    section = os.environ.get('SECTION')
    pipeline = [
        {'$match': {'PROPERTY_ID_NMA': {'$in': properties}}},
        {
            '$group': {
                '_id': '$CITY_NAME',
                'county': {'$first': '$COUNTY_NAME'},
                'city': {'$first': '$CITY_NAME'},
                'sum_area': {'$sum': '$UFS_TOTAL'},
                'count': {'$sum': 1}
            }
        }, 
        {
            '$group': {
                '_id': '$county',
                'county': {'$first': '$county'},
                'area_by_city': {'$push': {'name': '$city', 'value': '$sum_area'}},
                'value': {'$sum': '$count'},
                'area_per_county': {'$sum': '$sum_area'}
            }
        }, 
        {
            '$group': {
                '_id': None,
                'count_per_county': {'$push': {'name': '$county', 'value': '$value'}},
                'area_by_city': {'$push': '$area_by_city'},
                'total_area': {'$sum': '$area_per_county'},
                'total_property': {'$sum': '$value'}
            }
        }, 
        {
            '$project': {
                '_id': 0,
                'count_per_county': 1,
                'area_by_city': {
                    '$reduce': {
                        'input': '$area_by_city',
                        'initialValue': [],
                        'in': {'$concatArrays': ['$$this', '$$value']}
                    }
                },
                'total_area': 1,
                'total_property': 1
            }
        }
    ]
    
    with MongoConnectionManager(database, f'{section}_collection') as collection:
        data = list(collection.aggregate(pipeline=pipeline, allowDiskUse=True))[0]
    
    processed_data = process_output(data)
    return processed_data

In [21]:
with open('chart-parameters.json', 'r') as reader:
    ids = json.loads(reader.read())['ids']

In [24]:
data = prepare_chart_data(ids)
pprint(data)

{'area_by_city': [{'name': 'Sandnes', 'value': 26957.2},
                  {'name': 'Hå', 'value': 0.0},
                  {'name': 'Hjelmeland', 'value': 1504.0},
                  {'name': 'Stavanger', 'value': 8878.0},
                  {'name': 'Haugesund', 'value': 13882.0},
                  {'name': 'Bamble', 'value': 791.0},
                  {'name': 'Porsgrunn', 'value': 1820.0},
                  {'name': 'Vinje', 'value': 106.0},
                  {'name': 'Kragerø', 'value': 38002.2},
                  {'name': 'Færder', 'value': 72.0},
                  {'name': 'Tønsberg', 'value': 65.0},
                  {'name': 'Vågan', 'value': 473.0},
                  {'name': 'Kristiansand', 'value': 176.0},
                  {'name': 'Arendal', 'value': 6450.0},
                  {'name': 'Farsund', 'value': 604.0},
                  {'name': 'Ringsaker', 'value': 0.0},
                  {'name': 'Gran', 'value': 0.0},
                  {'name': 'Malvik', 'value': 21949.0},
    